# 3D CAE Model Humerus Labelmap


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from loguru import logger
import numpy as np
import time
import pickle
import h5py
from skimage.transform import downscale_local_mean,resize
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans,DBSCAN

import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import Conv3D,Conv3DTranspose,Dense,MaxPooling3D,UpSampling3D,Flatten,Dense,Reshape,GlobalAveragePooling3D,BatchNormalization,Dropout
from keras.activations import relu
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from itkwidgets import view

# My imports (se si modifica il file bisogna riavviare tutto il kernel)
from utilities import plot_all_slices_notzero,plot_slices,dice_coef

st=time.time()
with h5py.File("dataset_humerus_labelmap_res.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
st=time.time()
with h5py.File("dataset_humerus_labelmap_res_flipped.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
humerus_dataset.shape

## Downsampling

Meglio inziare con un downsampling per evitare problemi di memoria inizialmente.

Dataset: 288x210x393\
Padding a: 288x216x400\
Downsample per 4 a: 72x54x100

In [ ]:
humerus_dataset = np.pad(humerus_dataset,((0,0),(0,0),(3,3),(4,3)))

In [ ]:
humerus_dataset = resize(humerus_dataset,(humerus_dataset.shape[0],humerus_dataset.shape[1]/4,humerus_dataset.shape[2]/4,humerus_dataset.shape[3]/4),preserve_range=True,order=0,anti_aliasing=False)

In [ ]:
humerus_dataset.shape

In [ ]:
with h5py.File("resized_dataset_humerus_labelmap_res_flipped.hdf5","w") as f:
    f.create_dataset("mydataset",data=humerus_dataset)

# Model downsampled

Una volta importato il dataset possiamo iniziare ad occuparci della costruzione del modello. Partiamo da immagini downscalate

In [ ]:
st=time.time()
with h5py.File("resized_dataset_humerus_labelmap_res_flipped.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")
humerus_dataset.shape

In [ ]:
resized_dataset_exp = np.expand_dims(humerus_dataset,axis=-1)
resized_dataset_exp.shape

In [ ]:
# Model downscaled con STRIDE (meglio)
embedding_dim = 128
inputs = keras.Input(shape=(resized_dataset_exp.shape[1],resized_dataset_exp.shape[2],resized_dataset_exp.shape[3],1))

# Encoder
x = Conv3D(filters=32,kernel_size=(3,3,3),padding="same",name="conv1_1")(inputs)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="conv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=64,kernel_size=(3,4,3),padding="valid",name="conv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="conv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(4,3,3),padding="valid",name="conv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=128,kernel_size=(3,3,3),strides=2,padding="same",name="conv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(3,3,3),padding="valid",name="conv4")(x)
x = BatchNormalization()(x)
x = relu(x)

# Bottleneck
shape_before_encoding = x.shape[1:]
x = Flatten(name="flatten")(x) 
flattened_size = x.shape[1]
x = Dropout(0.3)(x)
embedding = Dense(embedding_dim,activation="relu",name="embedding")(x)
x = Dropout(0.3)(embedding)
x = Dense(flattened_size,activation="relu",name="expanding")(x)
x = Reshape(shape_before_encoding,name="reshape")(x)

# Decoder
x = Conv3DTranspose(filters=128,kernel_size=(3,3,3),padding="valid",name="deconv4")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),padding="same",name="deconv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="deconv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=32,kernel_size=(4,3,3),padding="valid",name="deconv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="deconv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=16,kernel_size=(3,4,3),padding="valid",name="deconv1_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=16,kernel_size=(3,3,3),strides=2,padding="same",name="deconv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

outputs = Conv3DTranspose(filters=1,kernel_size=(3,3,3),padding="same",name="deconv0",activation="sigmoid")(x)

model_downsample = keras.Model(inputs,outputs,name="model_downscaled")
model_downsample.summary()

with open("training_history/hum_loss.pkl","rb") as f1,open("training_history/hum_val_loss.pkl","rb") as f2,open("training_history/hum_dice.pkl","rb") as f3,open("training_history/hum_val_dice.pkl","rb") as f4:
    tot_loss = pickle.load(f1)
    tot_val_loss = pickle.load(f2)
    tot_dice = pickle.load(f3)
    tot_val_dice = pickle.load(f4)

In [ ]:
tot_loss = []
tot_val_loss = []
tot_dice = []
tot_val_dice = []

In [ ]:
model_downsample.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[dice_coef]
)

checkpoint = ModelCheckpoint(
    "models_weights/humerus_flipped_checkpoint.weights.h5",
    monitor="loss",
    save_best_only=True,
    save_weights_only=True
)

history = model_downsample.fit(
    x=resized_dataset_exp,
    y=resized_dataset_exp,
    callbacks=[checkpoint],
    # validation_split=0.2,
    batch_size=16,
    epochs=50,
    verbose=1,
)

In [ ]:
model_downsample.save_weights("models_weights/humerus_flipped_loss0084_250ep_noval.weights.h5")

In [ ]:
# Non memorizzo le ultime 50 epoche senza validation 
tot_loss += history.history['loss'] # Per appendere liste tra loro
tot_val_loss+=history.history['val_loss']
tot_dice+=history.history['dice_coef']
tot_val_dice+=history.history['val_dice_coef']

In [ ]:
with open("training_history/hum_flip_loss.pkl","wb") as f1,open("training_history/hum_flip_val_loss.pkl","wb") as f2,open("training_history/hum_flip_dice.pkl","wb") as f3,open("training_history/hum_flip_val_dice.pkl","wb") as f4:
    pickle.dump(tot_loss,f1)
    pickle.dump(tot_val_loss,f2)
    pickle.dump(tot_dice,f3)
    pickle.dump(tot_val_dice,f4)

In [ ]:
# "Loss"
fig,ax = plt.subplots(2,1,figsize=(18,18))
start = 0
end = 200

# Loss
ax[0].plot(tot_loss[start:end+1],marker=".")
ax[0].plot(tot_val_loss[start:end+1],marker=".")
ax[0].set_title('Loss del Modello')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoche')
ax[0].set_xticks([x for x in range(0,end+1-start,5)])
ax[0].set_xticklabels([x for x in range(start,end+1,5)])
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[0].grid()

# Dice coefficient
ax[1].plot(tot_dice[start:end+1],marker=".")
ax[1].plot(tot_val_dice[start:end+1],marker=".")
ax[1].set_title('Dice Coefficient del Modello')
ax[1].set_ylabel('Dice Coefficient')
ax[1].set_xlabel('Epoche')
ax[1].set_xticks([x for x in range(0,end+1-start,5)])
ax[1].set_xticklabels([x for x in range(start,end+1,5)])
ax[1].legend(['Train', 'Validation'], loc='upper left')
ax[1].grid()

## Predictions

In [ ]:
model_downsample.load_weights("models_weights/humerus_flipped_loss0138_50ep.weights.h5")

In [ ]:
pred_ct = 400

In [ ]:
pred = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred_squeezed = np.squeeze(pred)
pred_squeezed_binary = np.where(pred_squeezed >= 0.5,1,0)

In [ ]:
dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred_squeezed)

In [ ]:
start_slice = 60
plot_slices(resized_dataset_exp[pred_ct],2,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred_squeezed_binary,2,8,start_slice,title="Predizione Binaria",titlesize=40)
plot_slices(pred_squeezed,2,8,start_slice,title="Predizione",titlesize=40)

In [ ]:
# Carico i pesi 50 epoche
model_downsample.load_weights("models_weights/humerus_loss0106_100ep.weights.h5")
pred1 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred1_squeezed = np.squeeze(pred1)
pred1_squeezed_binary = np.where(pred1_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred1_squeezed))

In [ ]:
# Carico i pesi 150 epoche
model_downsample.load_weights("models_weights/humerus_loss0092_150ep.weights.h5")
pred2 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred2_squeezed = np.squeeze(pred2)
pred2_squeezed_binary = np.where(pred2_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred2_squeezed))

In [ ]:
# Carico i pesi 200 epoche
model_downsample.load_weights("models_weights/humerus_loss0085_200ep.weights.h5")
pred3 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred3_squeezed = np.squeeze(pred3)
pred3_squeezed_binary = np.where(pred3_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred3_squeezed))

In [ ]:
start_slice = 60
plot_slices(resized_dataset_exp[pred_ct],1,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred1_squeezed_binary,1,8,start_slice,title="Predizione dopo 100 epoche",titlesize=40)
plot_slices(pred2_squeezed_binary,1,8,start_slice,title="Predizione dopo 150 epoche",titlesize=40)
plot_slices(pred3_squeezed_binary,1,8,start_slice,title="Predizione dopo 200 epoche",titlesize=40)

## Feature extraction

In [ ]:
model_downsample.load_weights("models_weights/humerus_checkpoint.weights.h5") # Il checkpoint contiene quello con loss minima

In [ ]:
feature_model = keras.Model(model_downsample.inputs,model_downsample.get_layer("embedding").output)

In [ ]:
features = feature_model.predict(resized_dataset_exp)

In [ ]:
features.shape

In [ ]:
with open(f"processing/humerus_res_flip_features_250ep.pkl","wb") as f:
    pickle.dump(features,f)

## Visualization (da qui su altro file)

In [ ]:
pca = PCA() # Prova anche con tutte le dimensioni
pca.fit(features)
cumulative_sum = pca.explained_variance_ratio_.cumsum() # Per vedere quanta varianza catturo e scegliere il numero di PC
fig, ax = plt.subplots()
sns.lineplot(cumulative_sum,marker="o")
ax.set_title("Cumulative sum of variance ratio")
ax.set_xlim(0,10)
print(cumulative_sum[:10])
# Catturo già il 94% della varianza con 4 PC

In [ ]:
num_components = 4
st = time.time()
features_pca = PCA(n_components=num_components).fit_transform(features) # Userò anche per il clustering
features_tsne = TSNE(n_iter=1000,verbose=1).fit_transform(features_pca) # Per la visualizzazione in 2D
print(f"Tempo dimensionality reduction: {time.time()-st:.2f} sec")

In [ ]:
print(features.shape)
print(features_pca.shape)
print(features_tsne.shape)

In [ ]:
data_scatter={}
data_scatter["dim1"] = features_tsne[:,0]
data_scatter["dim2"] = features_tsne[:,1]

In [ ]:
# sns.set(rc = {'figure.figsize':(20, 15)})
plt.figure(figsize=(15,10))
sns.scatterplot(
    data = data_scatter,
    x = "dim1",
    y = "dim2",
)

## Clustering

In [ ]:
# Stampo l'inerzia (distanza within cluster) per vari numeri di cluster per vedere qual è il numero di cluster ottimale (gomito)
distortions = []
sil = []
dbs = []
chi = []

for k in range(1,10):
    kmeanModel = KMeans(n_clusters=k,n_init="auto")
    # kmeanModel.fit(features_pca)
    labels = kmeanModel.fit_predict(features_pca)
    distortions.append(kmeanModel.inertia_)
    if k > 1:
        sil.append(silhouette_score(features_pca,labels))
        dbs.append(davies_bouldin_score(features_pca,labels)) 
        chi.append(calinski_harabasz_score(features_pca,labels)) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
sns.lineplot(x=range(2,10),y=sil,ax=ax[0],marker="o")
ax[0].set_title("Silhouette")  # Da -1 a 1, meglio 1, distanze intra e inter cluster medie
sns.lineplot(x=range(2,10),y=dbs,ax=ax[1],marker="o")
ax[1].set_title("Davies-Bouldin") # Più basso è meglio è, distanza intra cluster bassa e inter alta porta ad un indice basso
sns.lineplot(x=range(2,10),y=chi,ax=ax[2],marker="o")
ax[2].set_title("Calinski and Harabasz") 
# Più alto è meglio è, basato su distanze dei punti dentro un cluster dal centroide e distanze dei centroidi dei cluster
# dal centroide globale

In [ ]:
sns.lineplot(x=range(1,10),y=distortions,marker="o")
plt.title("Inertia")

Tutto sembra indicare che il numero ideale di cluster sia 3

In [ ]:
# Il gomito è intorno a 3-4-5 cluster
number_clusters = 3
kmeans = KMeans(n_clusters=number_clusters,n_init="auto")
labels = kmeans.fit_predict(features_pca) # KMeans non su 150 features per curse of dimensionality

In [ ]:
#dbscan = DBSCAN(eps=3,min_samples=5)
#labels = dbscan.fit_predict(features_pca)

In [ ]:
data_scatter["labels"] = labels

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    data = data_scatter,
    x = "dim1",
    y = "dim2",
    hue= "labels",
    palette=sns.color_palette("bright",number_clusters)
)
# Visualizzare con visualizzatore apposta cliccabile

## Heatmap

Coregistrazione CT mutual information

In [ ]:
# Provo ad ottenre una heatmap dell'ultimo livello dell'encoder per vedere dove si concentra l'estrazione delle feature

image_n = 110
conv_layer = model_downsample.get_layer("conv4")
heatmap_model = keras.Model(inputs=model_downsample.input, outputs=[conv_layer.output,model_downsample.output])
in_img = tf.Variable(np.expand_dims(resized_dataset_exp[image_n],axis=0),dtype=tf.float32)

with tf.GradientTape() as gtape:
    gtape.watch(in_img)
    heatmap_output, prediction = heatmap_model(in_img) # È come predict, ma da usare quando voglio differenziarlo (come qui)
    
grads = gtape.gradient(prediction,heatmap_output)
chan_importance = tf.reduce_mean(grads,axis=(0,1,2,3)) # Calcolo un valore che indica l'importanza di ogni canale

heatmap = tf.reduce_mean(heatmap_output*chan_importance,axis=-1)
print(np.min(heatmap),np.max(heatmap))

# Per normalizzare riduco il minimo a 0 e normalizzo tra 0 e 1
heatmap = tf.maximum(abs(heatmap),0) # Son tutti negativi, metto abs
print(np.min(heatmap),np.max(heatmap))

heatmap /= tf.reduce_max(heatmap)  # Normalizzo la heatmap tra 0 e 1
print(np.min(heatmap),np.max(heatmap))

In [ ]:
heatmap_output.shape,prediction.shape,grads.shape,chan_importance.shape,heatmap.shape

In [ ]:
plot_all_slices_notzero(np.squeeze(heatmap_output)[:,:,:,0])

In [ ]:
heatmap.shape

In [ ]:
heatmap = np.squeeze(np.asarray(heatmap))
heatmap_res = resize(heatmap,output_shape=(in_img.shape[1],in_img.shape[2],in_img.shape[3]))
plot_all_slices_notzero(heatmap_res)
# heat_opencv = cv2.resize(heatmap,(in_img.shape[1],in_img.shape[2],in_img.shape[3]))

In [ ]:
ct_slice = 70

In [ ]:
heatmap_res.shape[2]

In [ ]:
superimposed_images = []
for ct_slice in range(heatmap_res.shape[2]): # Le slice
    heat_img = np.uint8(heatmap_res[:,:,ct_slice]*255) # Moltiplico per 255 per visualizzazione
    in_img_cv = np.uint8(np.where(np.squeeze(in_img)[:,:,ct_slice] == 0,255,0)) # Cambio colori foreground e background per visualizzare meglio la heatmap 
    heat_img = cv2.applyColorMap(heat_img, cv2.COLORMAP_JET)
    in_img_cv = cv2.applyColorMap(in_img_cv, cv2.COLORMAP_BONE)
    superimposed_img = heat_img*0.9 + in_img_cv*0.7
    #plt.imshow(np.clip(superimposed_img,0,255).astype("uint8"))
    superimposed_images.append(np.clip(superimposed_img,0,255).astype("uint8"))

In [ ]:
superimposed_images = np.asarray(superimposed_images,dtype=np.uint8)
nrows = 16
ncols = 8
fig,ax = plt.subplots(
    nrows,
    ncols,
    figsize = (40,int(40/ncols)*nrows),
    constrained_layout=True
)
for i in range(nrows):
    for j in range(ncols):
        index = i*ncols+j
        ax[i,j].imshow(superimposed_images[index])
        ax[i,j].axis("off")

In [ ]:
#with tf.device("/GPU:0"):
inputs = keras.Input(shape=(humerus_dataset.shape[1],humerus_dataset.shape[2],humerus_dataset.shape[3],1))

# Encoder
x = Conv3D(filters=16,kernel_size=(4,4,3),activation="relu",padding="valid",name="conv1")(inputs)
x = MaxPooling3D(pool_size=(2,2,2),name="pool1")(x)

x = Conv3D(filters=32,kernel_size=(3,4,3),activation="relu",padding="valid",name="conv2")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool2")(x)

x = Conv3D(filters=64,kernel_size=(3,3,4),activation="relu",padding="valid",name="conv3")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool3")(x)

x = Conv3D(filters=128,kernel_size=(3,4,3),activation="relu",padding="valid",name="conv4")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool4")(x)

shape_before_encoding = x.shape[1:]
print(shape_before_encoding)
# Feature extraction
x = Flatten(name="flatten")(x) 
flattened_size = x.shape[1]
print(flattened_size)
embedding = Dense(100,activation="relu",name="embedding")(x)
x = Dense(flattened_size,activation="relu",name="expanding")(embedding)
x = Reshape(shape_before_encoding,name="reshape")(x)

# Decoder
x = Conv3DTranspose(filters=64,kernel_size=(3,3,4),activation="relu",padding="same",name="deconv4")(x)
x = UpSampling3D((2,2,2),name="upsample4")(x)

x = Conv3DTranspose(filters=64,kernel_size=(3,4,3),activation="relu",padding="valid",name="deconv3")(x)
x = UpSampling3D((2,2,2),name="upsample3")(x)

x = Conv3DTranspose(filters=32,kernel_size=(3,3,4),activation="relu",padding="valid",name="deconv2")(x)
x = UpSampling3D((2,2,2),name="upsample2")(x)

x = Conv3DTranspose(filters=16,kernel_size=(3,4,3),padding="valid",name="deconv1")(x)
x = UpSampling3D((2,2,2),name="upsample1")(x)

outputs = Conv3DTranspose(filters=1,kernel_size=(4,4,3),padding="valid",name="deconv0",activation="sigmoid")(x)

model = keras.Model(inputs,outputs,name="model_1_200")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.Accuracy()]
)

In [ ]:
print(dataset_1_200.shape)
in_shape = (humerus_dataset.shape[1],humerus_dataset.shape[2],humerus_dataset.shape[3],1)
dataset_1_200_expanded = np.expand_dims(humerus_dataset[:100],axis=-1) # solo i primi 100
dataset_1_200_expanded.shape

In [ ]:
#with tf.device("/GPU:0"):
model.fit(
    x=dataset_1_200_expanded,
    y=dataset_1_200_expanded,
    batch_size=2,
    epochs=2,
    verbose=1,
)

## Predictions

In [ ]:
pred_ct = 130

In [ ]:
pred = model_downsample.predict(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred_squeezed = np.squeeze(pred)
pred_squeezed_binary = np.where(pred_squeezed >= 0.5,1,0)

In [ ]:
np.min(pred)

In [ ]:
start_slice = 55
plot_slices(resized_dataset_exp[pred_ct],3,8,start_slice)
plot_slices(pred_squeezed_binary,3,8,start_slice)

In [ ]:
plot_all_slices_notzero(resized_dataset_exp[pred_ct])
plot_all_slices_notzero(pred_squeezed_binary)